In [1]:

from Experiment import Experiment
import matplotlib.pyplot as plt
%matplotlib qt

loadir = 'H:/My Drive/dark 2022/csv_dark/' # dir to save the data frame in # H:\My Drive\Ronis Exp\sagiv\data_analysis
exp_name = ['manipulated_2022_01_31_dark']#['2022_02_03','2022_03_10','2022_05_19','2022_03_03'] # 22_11_28
ref_frame = 0

experiment = Experiment(loadir,exp_name[0])


In [3]:
fig = experiment.plot_movies('time','pitch_y_frame', case = 'plot_mov', fig = False, mov = False)
fig.show()
fig = experiment.plot_movies('time','pitch_y_frame', case = 'plot_exp', fig = False, mov = False)
fig.show()
fig = experiment.plot_movies('time','pitch_body', case = 'plot_mov', fig = False, mov = False)
fig.show()

fig = experiment.plot_movies('time','CM_xy', case = 'plot_mov', fig = False, mov = False)
fig.show()

In [4]:
experiment.exp_dict['mov1'].header['body']

KeyError: 'mov1'

In [ ]:
experiment.exp_dict['mov1'].data['body'][]

In [6]:

fig = experiment.plot_movies('time','pitch_y_frame', case = 'plot_mov', fig = False, mov = False)
fig.show()


fig = experiment.plot_movies('time','pitch_y_frame_dot', case = 'plot_mov', fig = False, mov = False)
fig.show()

In [12]:

fig = experiment.plot_movies('time','yaw_z_frame', case = 'plot_mov', fig = False, mov = False)
fig.show()
fig = experiment.plot_movies('time','pitch_y_frame', case = 'plot_mov', fig = False, mov = False)
fig.show()
fig = experiment.plot_movies('time','roll_x_frame', case = 'plot_mov', fig = False, mov = False)
fig.show()

In [6]:

fig = experiment.plot_movies('time','CM_xy_dot_dot', case = 'plot_mov', fig = False, mov = False)
fig.show()

In [32]:
experiment.exp_dict[mov].data['body'][:,25] - experiment.exp_dict[mov].data['body'][:,25][experiment.exp_dict[mov].ref_frame]

array([-0.01120494108875665, -0.01091411000928482, -0.01062909751261977,
       ..., 0.034560426105704624, 0.03410571371421179,
       0.03364042511486203], dtype=object)

In [19]:
 [['yaw_z_frame','pitch_y_frame','roll_x_frame'] * [deriv] for deriv in ['','_dot','_dot_dot']]

TypeError: can't multiply sequence by non-int of type 'list'

In [15]:
import numpy as np
from scipy.signal import argrelextrema, savgol_filter,find_peaks

idx = experiment.exp_dict['mov1'].header['body']['pitch_y_frame']
for mov in experiment.exp_dict.keys():

   experiment.exp_dict[mov].data['body']=np.hstack((experiment.exp_dict[mov].data['body'], (experiment.exp_dict[mov].data['body'][:,25] - experiment.exp_dict[mov].data['body'][experiment.exp_dict[mov].ref_frame,25])[np.newaxis,:].T))
   experiment.exp_dict[mov].header['body']['cm_test_dot'] = experiment.exp_dict[mov].data['body'].shape[1] - 1
fig = experiment.plot_movies('time','cm_test_dot', case = 'plot_mov', fig = False, mov = False)
fig.show()

IndexError: index 25 is out of bounds for axis 1 with size 24